# 1.0. Baseline Setup

Since this competition has blocked late submissions, this notebook constructs a test dataset based on the latest dataset released by the host. Additionally, it pre-implements the evaluation metric, SMAPE, so that it can be used consistently across all baseline notebooks.

1. Pre-implements the evaluation metric, SMAPE.
2. Constructs and Store a test dataset.

In [1]:
import os, datetime
from tqdm import tqdm

import numpy as np
import pandas as pd

Load Data to figure out the test period

In [2]:
data_dir = '../data/'

df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
df_test = pd.read_csv(os.path.join(data_dir, 'test.csv'))
df_revealed_test = pd.read_csv(os.path.join(data_dir, 'revealed_test.csv'))
df_submission = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))
df_census = pd.read_csv(os.path.join(data_dir, 'census_starter.csv'))

In [3]:
df_train.info() # 122,265 entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122265 entries, 0 to 122264
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   row_id                 122265 non-null  object 
 1   cfips                  122265 non-null  int64  
 2   county                 122265 non-null  object 
 3   state                  122265 non-null  object 
 4   first_day_of_month     122265 non-null  object 
 5   microbusiness_density  122265 non-null  float64
 6   active                 122265 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 6.5+ MB


In [4]:
print(f'unique counties: {df_train.cfips.nunique()}')

unique counties: 3135


In [5]:
# convert first_day_of_month from str to datetime
df_train.first_day_of_month = pd.to_datetime(df_train.first_day_of_month)
df_test.first_day_of_month = pd.to_datetime(df_test.first_day_of_month)
df_revealed_test.first_day_of_month = pd.to_datetime(df_revealed_test.first_day_of_month)

strformat = '%Y-%m'
print(f'train period:		{df_train.first_day_of_month.min().strftime(strformat)} ~ {df_train.first_day_of_month.max().strftime(strformat)}')
print(f'test period: 		{df_test.first_day_of_month.min().strftime(strformat)} ~ {df_test.first_day_of_month.max().strftime(strformat)}')
print(f'revealed period: 	{df_revealed_test.first_day_of_month.min().strftime(strformat)} ~ {df_revealed_test.first_day_of_month.max().strftime(strformat)}')

train period:		2019-08 ~ 2022-10
test period: 		2022-11 ~ 2023-06
revealed period: 	2022-11 ~ 2022-12


According to the data description for this competition,

> `revealed_test.csv` During the submission period, only the most recent month of data will be used for the public leaderboard. Any test set data older than that will be published in `revealed_test.csv`, closely following the usual data release cycle for the microbusiness report. We expect to publish one copy of `revealed_test.csv` in mid February. This file's schema will match train.csv.

- Only the most recent month of data was used for the public leaderboard.
- `revealed_test.csv` was the only hint of test data during the competition, so the competitors likely had no visibility into the 2023 data.
- The private leaderboard was calculated with approximately 88% of the test data (as described in the private leaderboard instructions).

## 1. Pre-implements the evaluation metric, SMAPE.

Technically, [SMAPE](https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error) cannot be calculated when both the predicted and actual value are 0 (zero). However, in this competition, the host defined SMAPE = 0 in case where the actual and predicted values are both 0.

In [6]:
# define evaluation metric, SMAPE
def SMAPE(actuals, preds):
	smape = 0
	for actual, pred in zip(actuals, preds):
		if (actual==0) and (pred==0): continue
		smape += abs(actual-pred) / (abs(actual) + abs(pred)) * 2 * 100
	return smape / len(actuals)

# Example data
data = {
    'date': pd.date_range(start='2020-01-01', periods=10, freq='D'),
    'actual': [100, 150, 200, 250, 300, 350, 400, 450, 500, 550],
    'forecast': [110, 140, 210, 260, 295, 340, 410, 460, 480, 570]
}
df = pd.DataFrame(data)

SMAPE(df['actual'], df['forecast']) # it's supposed to be 4.21 %

4.211920087419261

## 2. Constructs and Store a test dataset

Since today is 2024, Godaddy already released Microbusiness Density data from Aug 2019 - Jun 2024. Among four countries, we sole need States data from [this link](https://www.godaddy.com/ventureforward/resource/united-states-microbusiness-density/). I manually downloaded this data on web browser, and added `VF_US_MD_CFIPS_Q224_CSV.csv` file to `godaddy/data` directory. This will be the source of test dataset.

Download info
- Microbusiness Density
- Date Range: Aug 2019 - Jun 2024
- download this data with browser to data/
- godaddy_us_data = 'https://www.godaddy.com/ventureforward/resource/united-states-microbusiness-density/vf_us_md_q224/'

In [51]:
# load data
df_godaddy = pd.read_csv(os.path.join(data_dir, 'VF_US_MD_CFIPS_Q224_CSV.csv'))
df_godaddy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3415 entries, 0 to 3414
Columns: 137 entries, cfips to new_md_jun24
dtypes: float64(134), object(3)
memory usage: 3.6+ MB


In [52]:
# filter valid cfips and columns
df_godaddy = df_godaddy[df_godaddy.cfips.isin(df_test.cfips)]
df_godaddy = df_godaddy[['cfips'] + target_cols]

# melt columns into a column
df_godaddy = df_godaddy.melt(id_vars=['cfips'], var_name='month', value_name='microbusiness_density')

df_godaddy.head()

,cfips,month,microbusiness_density
0,1001.0,md_nov22,3.292227
1,1003.0,md_nov22,7.941944
2,1005.0,md_nov22,1.235309
3,1007.0,md_nov22,1.275281
4,1009.0,md_nov22,1.803057


In [53]:
# select target data: 2022-11 ~ 2023-06
target_cols = [
	'md_nov22', 'md_dec22', 
	'md_jan23', 'md_feb23', 'md_mar23', 'md_apr23', 'md_may23', 'md_jun23'
]
target_cols_num = [
	'2022-11-01', '2022-12-01', 
	'2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01'
]

raw2num = {col:colnum for col, colnum in zip(target_cols, target_cols_num)}
df_godaddy.month = df_godaddy.month.map(raw2num)

In [54]:
# generate row_id column
df_godaddy['row_id'] = df_godaddy.cfips.astype(int).astype(str) + '_' + df_godaddy.month
df_godaddy = df_godaddy[['row_id', 'microbusiness_density']]

# sort new test data along the order of submission
sortmap = {val:idx for idx, val in enumerate(df_submission.row_id.values)}
df_godaddy['sortmap'] = df_godaddy.row_id.map(sortmap)
df_godaddy = df_godaddy.sort_values('sortmap').drop(columns='sortmap')

In [57]:
# store data
df_godaddy.to_csv('../data/all_revealed_test.csv', index=False)

Demo Evaluation with submission file and test file

In [58]:
smape_value = SMAPE(df_godaddy.microbusiness_density.values, df_submission.microbusiness_density.values).item()
smape_value

59.62397976302414

FYI, private LB (88% of the test data)
- 1st place: 3.7877
- 10st place: 3.8827
- 354st place: 4.1288